In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


sklearn version:  1.0.2
TF version:  2.8.0
GPU installed:  True
4 Physical GPUs, 4 Logical GPUs


2022-06-05 09:25:17.156991: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-05 09:25:19.689376: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7482 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2022-06-05 09:25:19.690251: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 7397 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-06-05 09:25:19.691032: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU

In [2]:
X_train=np.uint8(np.load('./X_train_aug.npy'))
y_train=np.uint8(np.load('./y_train_aug.npy'))
X_valid=np.uint8(np.load('./X_valid.npy'))
y_valid=np.uint8(np.load('./y_valid.npy'))

In [3]:
X_valid=tf.reshape(X_valid,[-1,28,28,1])
X_valid_resize=np.uint8(tf.image.resize(X_valid, [32, 32]))
X_train.shape,X_valid_resize.shape,X_train.dtype

2022-06-05 09:25:26.032266: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 671805440 exceeds 10% of free system memory.


((1074263, 32, 32, 1), (164015, 32, 32, 1), dtype('uint8'))

In [4]:
#황성현
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, 
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME", activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [18]:
    #황성현 전민재
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.0003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_8 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [19]:
#전민재
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

In [20]:
#황성현 전민재
from datetime import datetime
K = keras.backend
logs = "logs/" + datetime.now().strftime("SMV2L_lrsch_onecycle_aug128%Y%m%d-%H%M%S")
onecycle = OneCycleScheduler(len(X_train) // batch_size * 50, max_rate=0.003)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_adam_best_lrsch_onecycle_aug128.h5",save_best_only=True)

history=smv2_l.fit(X_train,y_train,validation_data=(X_valid_resize,y_valid),batch_size = batch_size, epochs=50,callbacks=[tboard_callback, check_best_cb,onecycle])

2022-06-05 10:47:13.165515: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-05 10:47:13.165569: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-05 10:47:13.918482: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-05 10:47:13.918686: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed


Epoch 1/50
 498/8393 [>.............................] - ETA: 3:04 - loss: 3.6107 - accuracy: 0.1396

2022-06-05 10:47:29.045363: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-05 10:47:29.045419: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.


 519/8393 [>.............................] - ETA: 3:17 - loss: 3.5810 - accuracy: 0.1438

2022-06-05 10:47:30.629577: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-05 10:47:30.630992: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-05 10:47:30.811819: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 14617 callback api events and 14688 activity events. 
2022-06-05 10:47:31.064490: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-05 10:47:31.376836: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/SMV2L_lrsch_onecycle_aug12820220605-104713/plugins/profile/2022_06_05_10_47_31

2022-06-05 10:47:31.603151: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/SMV2L_lrsch_onecycle_aug12820220605-104713/plugins/profile/2022_06_05_10_47_31/a58a2589147d.trace.json.gz


 524/8393 [>.............................] - ETA: 3:39 - loss: 3.5737 - accuracy: 0.1450

2022-06-05 10:47:31.866005: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/SMV2L_lrsch_onecycle_aug12820220605-104713/plugins/profile/2022_06_05_10_47_31

2022-06-05 10:47:31.873432: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/SMV2L_lrsch_onecycle_aug12820220605-104713/plugins/profile/2022_06_05_10_47_31/a58a2589147d.memory_profile.json.gz
2022-06-05 10:47:31.878697: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/SMV2L_lrsch_onecycle_aug12820220605-104713/plugins/profile/2022_06_05_10_47_31
Dumped tool data for xplane.pb to logs/SMV2L_lrsch_onecycle_aug12820220605-104713/plugins/profile/2022_06_05_10_47_31/a58a2589147d.xplane.pb
Dumped tool data for overview_page.pb to logs/SMV2L_lrsch_onecycle_aug12820220605-104713/plugins/profile/2022_06_05_10_47_31/a58a2589147d.overview_page.pb
Dumped tool data for input_pipeline.pb to logs/SMV2L_lrs

8393/8393 [==============================] - 214s 25ms/step - loss: 0.9895 - accuracy: 0.6962 - val_loss: 0.4561 - val_accuracy: 0.8348
Epoch 2/50
8393/8393 [==============================] - 208s 25ms/step - loss: 0.4509 - accuracy: 0.8348 - val_loss: 0.4124 - val_accuracy: 0.8489
Epoch 3/50
8393/8393 [==============================] - 207s 25ms/step - loss: 0.4104 - accuracy: 0.8475 - val_loss: 0.3915 - val_accuracy: 0.8549
Epoch 4/50
8393/8393 [==============================] - 207s 25ms/step - loss: 0.3928 - accuracy: 0.8530 - val_loss: 0.3988 - val_accuracy: 0.8508
Epoch 5/50
8393/8393 [==============================] - 210s 25ms/step - loss: 0.3821 - accuracy: 0.8560 - val_loss: 0.4168 - val_accuracy: 0.8489
Epoch 6/50
8393/8393 [==============================] - 207s 25ms/step - loss: 0.3752 - accuracy: 0.8584 - val_loss: 0.3907 - val_accuracy: 0.8567
Epoch 7/50
8393/8393 [==============================] - 207s 25ms/step - loss: 0.3694 - accuracy: 0.8601 - val_loss: 0.3676 - val

In [21]:
smv2_l.save('./smv2l_adam_lrsch_onecycle_aug128.h5')